In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import matplotlib.pyplot as plt
from loguru import logger
from matbench.bench import MatbenchBenchmark
from matbench.constants import CLF_KEY

from gptchem.gpt_regressor import GPTRegressor
from gptchem.tuner import Tuner

logger.enable("gptchem")

In [13]:
import time

import decorator


def retry(howmany, *exception_types, **kwargs):
    timeout = kwargs.get("timeout", 0.0)  # seconds

    @decorator.decorator
    def tryIt(func, *fargs, **fkwargs):
        for _ in range(howmany):
            try:
                return func(*fargs, **fkwargs)
            except exception_types or Exception as e:
                print(e)
                if timeout is not None:
                    time.sleep(timeout)

    return tryIt

In [14]:
mb = MatbenchBenchmark(
    autoload=True,
    subset=[
          "matbench_expt_gap",
        #"matbench_steels",
    ],
)

2023-01-31 17:13:53 INFO     Initialized benchmark 'matbench_v0.1' with 1 tasks: 
['matbench_expt_gap']


In [15]:
@retry(3, timeout=5)
def train_test_fold(task, fold):
    regressor = GPTRegressor(
        task.metadata["target"], Tuner(n_epochs=8, learning_rate_multiplier=0.02, wandb_sync=False)
    )
    train_inputs, train_outputs = task.get_train_and_val_data(fold)

    # train and validate your model
    regressor.fit(train_inputs, train_outputs.values)

    # Get testing data
    test_inputs = task.get_test_data(fold, include_target=False)

    # Predict on the testing data
    # Your output should be a pandas series, numpy array, or python iterable
    # where the array elements are floats or bools
    predictions = regressor.predict(test_inputs)

    # Record your data!
    task.record(fold, predictions)
    return predictions

In [16]:
from collections import defaultdict

In [17]:
predictions = defaultdict(list)

for task in mb.tasks:
    task.load()

    for fold_ind, fold in enumerate(task.folds):
        if task.is_recorded[fold_ind]:
            print(f"Skipping fold {fold_ind} of {task.dataset_name}")
            continue
        pred = train_test_fold(task, fold)
        predictions[task.dataset_name].append(pred)
        train_inputs, train_outputs = task.get_train_and_val_data(fold)

    # print(f"{task.dataset_name}: MAE  {task.scores['mae']['mean']}")

2023-01-31 17:13:55 INFO     Dataset matbench_expt_gap already loaded; not reloading dataset.


Upload progress: 100%|██████████| 421k/421k [00:00<00:00, 707Mit/s]


Uploaded file from /Users/kevinmaikjablonka/git/kjappelbaum/gptchem/experiments/04_regression/matbench/out/20230131_171355/train.jsonl: file-u8iPFyFXJAp6Eo3zDKB3NlPS


2023-01-31 17:13:58.017 | DEBUG    | gptchem.tuner:tune:188 - Requested fine tuning. {
  "created_at": 1675181637,
  "events": [
    {
      "created_at": 1675181637,
      "level": "info",
      "message": "Created fine-tune: ft-LDIyknMulwVBINhl3kaxpTO2",
      "object": "fine-tune-event"
    }
  ],
  "fine_tuned_model": null,
  "hyperparams": {
    "batch_size": null,
    "learning_rate_multiplier": 0.02,
    "n_epochs": 8,
    "prompt_loss_weight": 0.01
  },
  "id": "ft-LDIyknMulwVBINhl3kaxpTO2",
  "model": "ada",
  "object": "fine-tune",
  "organization_id": "org-TFRJXw3PPQocOWbu71eI2t9U",
  "result_files": [],
  "status": "pending",
  "training_files": [
    {
      "bytes": 420861,
      "created_at": 1675181637,
      "filename": "/Users/kevinmaikjablonka/git/kjappelbaum/gptchem/experiments/04_regression/matbench/out/20230131_171355/train.jsonl",
      "id": "file-u8iPFyFXJAp6Eo3zDKB3NlPS",
      "object": "file",
      "purpose": "fine-tune",
      "status": "uploaded",
      "

2023-01-31 20:48:06 INFO     Recorded fold matbench_expt_gap-0 successfully.
Input contains NaN.


Upload progress: 100%|██████████| 421k/421k [00:00<00:00, 291Mit/s]


Uploaded file from /Users/kevinmaikjablonka/git/kjappelbaum/gptchem/experiments/04_regression/matbench/out/20230131_204811/train.jsonl: file-gGiKQ01RGBEBTLRdJGzVXr8n


2023-01-31 20:48:14.212 | DEBUG    | gptchem.tuner:tune:188 - Requested fine tuning. {
  "created_at": 1675194493,
  "events": [
    {
      "created_at": 1675194493,
      "level": "info",
      "message": "Created fine-tune: ft-1QMg9mMIOKUov8RWbGvpVB2m",
      "object": "fine-tune-event"
    }
  ],
  "fine_tuned_model": null,
  "hyperparams": {
    "batch_size": null,
    "learning_rate_multiplier": 0.02,
    "n_epochs": 8,
    "prompt_loss_weight": 0.01
  },
  "id": "ft-1QMg9mMIOKUov8RWbGvpVB2m",
  "model": "ada",
  "object": "fine-tune",
  "organization_id": "org-TFRJXw3PPQocOWbu71eI2t9U",
  "result_files": [],
  "status": "pending",
  "training_files": [
    {
      "bytes": 420861,
      "created_at": 1675194493,
      "filename": "/Users/kevinmaikjablonka/git/kjappelbaum/gptchem/experiments/04_regression/matbench/out/20230131_204811/train.jsonl",
      "id": "file-gGiKQ01RGBEBTLRdJGzVXr8n",
      "object": "file",
      "purpose": "fine-tune",
      "status": "uploaded",
      "

In [ ]:
task.scores

ValueError: Cannot score unless all folds are recorded!; folds [0, 1, 2, 3, 4] not recorded!

In [ ]:
mb.to_file("gpt_expt_gap_bench.json.gz")

2023-01-30 18:53:04 INFO     Successfully wrote MatbenchBenchmark to file 'gpt_steel_bench.json.gz'.
